In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.classical import expr
import numpy as np
from qiskit.quantum_info import Statevector


In [53]:
def create_circuit(N, theta, phi, lam):
    """Example distillation circuit template for 3 Bell pairs."""
    assert len(theta) + 1 == N
    assert len(phi) + 1 == N
    assert len(lam) + 1 == N

    target_a = N - 1
    target_b = N
    qr = QuantumRegister(N * 2, 'q')  
    cr = ClassicalRegister(N * 2 + 1, 'c')  # Classical bits for measurements + flag
    qc = QuantumCircuit(qr, cr)
        
    for i in range(N - 1):
        qc.u(theta[i], phi[i], lam[i], qr[i])
        qc.u(theta[i], -phi[i], -lam[i], qr[2 * N - 1 - i])
    

    for i in range(N - 1):
        qc.u(theta[i], phi[i], lam[i], qr[N-1])
        qc.u(theta[i], -phi[i], -lam[i], qr[N])

        qc.cx(qr[target_a], qr[i])
        qc.cx(qr[target_b], qr[2 * N - 1 - i])
    
        qc.measure(qr[i], cr[i * 2 + 1])
        qc.measure(qr[2 * N - 1 - i], cr[i * 2 + 2])

        qc.u(-theta[i], -lam[i], -phi[i], qr[N-1])
        qc.u(-theta[i], lam[i], phi[i], qr[N])
        qc.barrier()
        


    for i in range(N - 1):
        qc.store(cr[i * 2 + 1], expr.bit_xor(cr[i * 2 + 1], cr[i * 2 + 2]))
        qc.store(cr[0], expr.bit_or(cr[0], cr[i * 2 + 1]))
        # qc.store(cr[0], expr.bit_not(cr[0]))
    
    return qc
circuit = create_circuit(2, [0], [0], [0])
print(circuit.draw(output='text'))

     ┌──────────┐┌───┐┌─┐             ░ 
q_0: ┤ U(0,0,0) ├┤ X ├┤M├─────────────░─
     ├──────────┤└─┬─┘└╥┘┌──────────┐ ░ 
q_1: ┤ U(0,0,0) ├──■───╫─┤ U(0,0,0) ├─░─
     ├──────────┤      ║ ├──────────┤ ░ 
q_2: ┤ U(0,0,0) ├──■───╫─┤ U(0,0,0) ├─░─
     ├──────────┤┌─┴─┐ ║ └───┬─┬────┘ ░ 
q_3: ┤ U(0,0,0) ├┤ X ├─╫─────┤M├──────░─
     └──────────┘└───┘ ║     └╥┘      ░ 
c: 5/══════════════════╩══════╩═════════
                       1      2         


In [54]:
from itertools import product
I_gate = [0, 0, 0]
H_gate = [np.pi/2, 0, np.pi]
X_gate = [np.pi/2, -np.pi/2, np.pi/2]

options = [I_gate, H_gate, X_gate]
names   = ["I", "H", "X"]   # optional, just for printing labels

# 9 circuits: (gate1, gate2) where each gate is in options
for (g1, g2), (n1, n2) in zip(product(options, repeat=2), product(names, repeat=2)):
    thetalist = [g1[0], g2[0]]
    philist   = [g1[1], g2[1]]
    lamlist   = [g1[2], g2[2]]
    circuit = create_circuit(3, thetalist, philist, lamlist)
    drawing_one_line = circuit.draw(output="text").single_string()
    print(circuit.draw(output="text", fold=-1)) 

     ┌──────────┐┌───┐┌─┐             ░                                  ░ 
q_0: ┤ U(0,0,0) ├┤ X ├┤M├─────────────░──────────────────────────────────░─
     ├──────────┤└─┬─┘└╥┘             ░             ┌───┐┌─┐             ░ 
q_1: ┤ U(0,0,0) ├──┼───╫──────────────░─────────────┤ X ├┤M├─────────────░─
     ├──────────┤  │   ║ ┌──────────┐ ░ ┌──────────┐└─┬─┘└╥┘┌──────────┐ ░ 
q_2: ┤ U(0,0,0) ├──■───╫─┤ U(0,0,0) ├─░─┤ U(0,0,0) ├──■───╫─┤ U(0,0,0) ├─░─
     ├──────────┤      ║ ├──────────┤ ░ ├──────────┤      ║ ├──────────┤ ░ 
q_3: ┤ U(0,0,0) ├──■───╫─┤ U(0,0,0) ├─░─┤ U(0,0,0) ├──■───╫─┤ U(0,0,0) ├─░─
     ├──────────┤  │   ║ └──────────┘ ░ └──────────┘┌─┴─┐ ║ └───┬─┬────┘ ░ 
q_4: ┤ U(0,0,0) ├──┼───╫──────────────░─────────────┤ X ├─╫─────┤M├──────░─
     ├──────────┤┌─┴─┐ ║     ┌─┐      ░             └───┘ ║     └╥┘      ░ 
q_5: ┤ U(0,0,0) ├┤ X ├─╫─────┤M├──────░───────────────────╫──────╫───────░─
     └──────────┘└───┘ ║     └╥┘      ░                   ║      ║       ░ 
c: 7/═══════

In [26]:
# Bell state basis vectors
bell_states = {
    'phi+': np.array([1, 0, 0, 1]) / np.sqrt(2),   # (|00⟩ + |11⟩)/√2
    'phi-': np.array([1, 0, 0, -1]) / np.sqrt(2),  # (|00⟩ - |11⟩)/√2
    'psi+': np.array([0, 1, 1, 0]) / np.sqrt(2),   # (|01⟩ + |10⟩)/√2
    'psi-': np.array([0, 1, -1, 0]) / np.sqrt(2),  # (|01⟩ - |10⟩)/√2
}

In [44]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator

def analyze(f, a, b, c, circuit):
    # Interpret f,a,b,c as PROBABILITY weights for a Bell-diagonal mixture
    total = f + a + b + c
    if total <= 0:
        raise ValueError("f+a+b+c must be > 0")
    pf, pa, pb, pc = f/total, a/total, b/total, c/total

    # Bell kets (length-4)
    k_phi_p = bell_states['phi+']
    k_phi_m = bell_states['phi-']
    k_psi_p = bell_states['psi+']
    k_psi_m = bell_states['psi-']

    # 2-qubit density matrix for one pair (Bell-diagonal)
    rho_pair = (
        pf * np.outer(k_phi_p, k_phi_p.conj()) +
        pa * np.outer(k_phi_m, k_phi_m.conj()) +
        pb * np.outer(k_psi_p, k_psi_p.conj()) +
        pc * np.outer(k_psi_m, k_psi_m.conj())
    )

    # Build 6-qubit rho for 3 independent pairs, embedded into qubits:
    # (q0,q5), (q1,q4), (q2,q3)
    # We'll construct rho_6q by iterating computational basis indices.
    rho_6q = np.zeros((64, 64), dtype=complex)

    # Precompute pair-basis mapping: pair_index = q_low + 2*q_high
    # For your pairs: (q0,q5), (q1,q4), (q2,q3)
    for q0 in range(2):
        for q1 in range(2):
            for q2 in range(2):
                for q3 in range(2):
                    for q4 in range(2):
                        for q5 in range(2):
                            i = q5*32 + q4*16 + q3*8 + q2*4 + q1*2 + q0
                            p0_i = q0 + 2*q5
                            p1_i = q1 + 2*q4
                            p2_i = q2 + 2*q3

                            for r0 in range(2):
                                for r1 in range(2):
                                    for r2 in range(2):
                                        for r3 in range(2):
                                            for r4 in range(2):
                                                for r5 in range(2):
                                                    j = r5*32 + r4*16 + r3*8 + r2*4 + r1*2 + r0
                                                    p0_j = r0 + 2*r5
                                                    p1_j = r1 + 2*r4
                                                    p2_j = r2 + 2*r3

                                                    rho_6q[i, j] = (
                                                        rho_pair[p0_i, p0_j] *
                                                        rho_pair[p1_i, p1_j] *
                                                        rho_pair[p2_i, p2_j]
                                                    )

    # Create circuit
    qr = QuantumRegister(6)
    cr = ClassicalRegister(7)
    qc = QuantumCircuit(qr, cr)

    # Initialize density matrix in Aer
    # This method name exists in many Aer versions; if yours errors, tell me the error text.
    qc.set_density_matrix(rho_6q)

    qc.compose(circuit, inplace=True)

    simulator = AerSimulator(method="density_matrix")
    shots = 100000
    counts = simulator.run(qc, shots=shots).result().get_counts()

    accepted = 0
    success = 0
    for key, value in counts.items():
        # NOTE: Qiskit count strings are ordered c[n-1]...c[0]
        # key[-1] is c0. Ensure your acceptance bit is actually c0.
        if key[-1] == '0':
            accepted += value
            # key[0], key[1] are c6 and c5. Ensure those are your "success" bits.
            if key[0] == '0' and key[1] == '0':
                success += value

    fidelity = (success / accepted) if accepted else 0.0
    success_rate = accepted / shots

    print(f"Fidelity: {fidelity:.4f}, Success Rate: {success_rate:.4f}")
    return fidelity, success_rate


In [45]:
import sqlite3
import json
import hashlib
from datetime import datetime

def init_db(db_path="results.sqlite"):
    con = sqlite3.connect(db_path)
    cur = con.cursor()

    cur.execute("""
    CREATE TABLE IF NOT EXISTS circuits (
        circuit_id TEXT PRIMARY KEY,
        label TEXT NOT NULL,
        qasm TEXT,
        meta_json TEXT
    )
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS runs (
        run_id INTEGER PRIMARY KEY AUTOINCREMENT,
        circuit_id TEXT NOT NULL,
        noise_label TEXT NOT NULL,
        f REAL NOT NULL,
        a REAL NOT NULL,
        b REAL NOT NULL,
        c REAL NOT NULL,
        fidelity REAL NOT NULL,
        success_rate REAL NOT NULL,
        created_at TEXT NOT NULL,
        FOREIGN KEY (circuit_id) REFERENCES circuits(circuit_id)
    )
    """)

    cur.execute("CREATE INDEX IF NOT EXISTS idx_runs_circuit ON runs(circuit_id)")
    cur.execute("CREATE INDEX IF NOT EXISTS idx_runs_noise ON runs(noise_label)")

    con.commit()
    return con

def circuit_fingerprint(circuit):
    """
    Stable ID so you can re-run experiments and append to same circuit entry.
    """
    try:
        qasm = circuit.qasm()
    except Exception:
        # If QASM export fails, fall back to repr
        qasm = repr(circuit)

    h = hashlib.sha256(qasm.encode("utf-8")).hexdigest()[:16]
    return h, qasm

def upsert_circuit(con, circuit, label, meta=None):
    circuit_id, qasm = circuit_fingerprint(circuit)
    meta_json = json.dumps(meta or {}, sort_keys=True)

    cur = con.cursor()
    cur.execute("""
        INSERT OR IGNORE INTO circuits (circuit_id, label, qasm, meta_json)
        VALUES (?, ?, ?, ?)
    """, (circuit_id, label, qasm, meta_json))
    con.commit()
    return circuit_id

def insert_run(con, circuit_id, noise_label, f, a, b, c, fidelity, success_rate):
    cur = con.cursor()
    cur.execute("""
        INSERT INTO runs (
            circuit_id, noise_label, f, a, b, c, fidelity, success_rate, created_at
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        circuit_id, noise_label, float(f), float(a), float(b), float(c),
        float(fidelity), float(success_rate),
        datetime.utcnow().isoformat(timespec="seconds") + "Z"
    ))
    con.commit()


In [46]:
from itertools import product
I_gate = [0, 0, 0]
H_gate = [np.pi/2, 0, np.pi]
X_gate = [np.pi/2, -np.pi/2, np.pi/2]

options = [I_gate, H_gate, X_gate]
names   = ["I", "H", "X"]   # optional, just for printing labels

# 9 circuits: (gate1, gate2) where each gate is in options
for (g1, g2), (n1, n2) in zip(product(options, repeat=2), product(names, repeat=2)):
    thetalist = [g1[0], g2[0]]
    philist   = [g1[1], g2[1]]
    lamlist   = [g1[2], g2[2]]

    circuit = create_circuit(3, thetalist, philist, lamlist)
    analyze(1, 0, 0, 0, circuit)
    analyze(0.9, 0, 0, 0.3, circuit) # psi-
    analyze(0.9, 0, 0.3, 0, circuit) # psi+
    analyze(0.9, 0.3, 0, 0, circuit) # phi-
    analyze(0.9, 0.3/np.sqrt(3), 0.3/np.sqrt(3), 0.3/np.sqrt(3), circuit) # even mix of all 3

Fidelity: 1.0000, Success Rate: 1.0000
Fidelity: 1.0000, Success Rate: 0.4349
Fidelity: 1.0000, Success Rate: 0.4352
Fidelity: 1.0000, Success Rate: 1.0000
Fidelity: 1.0000, Success Rate: 0.4465
Fidelity: 1.0000, Success Rate: 1.0000
Fidelity: 1.0000, Success Rate: 0.4711
Fidelity: 1.0000, Success Rate: 0.6247
Fidelity: 1.0000, Success Rate: 0.5613
Fidelity: 1.0000, Success Rate: 0.3845
Fidelity: 1.0000, Success Rate: 1.0000
Fidelity: 1.0000, Success Rate: 0.5631
Fidelity: 1.0000, Success Rate: 0.4350
Fidelity: 1.0000, Success Rate: 0.5636
Fidelity: 1.0000, Success Rate: 0.3805
Fidelity: 1.0000, Success Rate: 1.0000
Fidelity: 1.0000, Success Rate: 0.4711
Fidelity: 1.0000, Success Rate: 0.5603
Fidelity: 1.0000, Success Rate: 0.6256
Fidelity: 1.0000, Success Rate: 0.3854
Fidelity: 1.0000, Success Rate: 1.0000
Fidelity: 1.0000, Success Rate: 0.4388
Fidelity: 1.0000, Success Rate: 1.0000
Fidelity: 1.0000, Success Rate: 0.4377
Fidelity: 1.0000, Success Rate: 0.4439
Fidelity: 1.0000, Success

In [6]:
# Test 2: phi+ with psi+ noise (should be suppressed)
print("TEST 2: psi+ noise")
analyze(0.9, 0, 0.3, 0)

TEST 2: psi+ noise
Fidelity: 0.9987, Success Rate: 0.7304


In [11]:
# Test 3: phi+ with psi- noise (should be suppressed)
print("TEST 3: psi- noise")
analyze(0.9, 0, 0, 0.3)

TEST 3: psi- noise
Fidelity: 0.7547, Success Rate: 1.0000


In [8]:
# Test 4: phi+ with phi- noise (should NOT be suppressed)
print("TEST 4: phi- noise")
analyze(0.9, 0.3, 0, 0)

TEST 4: phi- noise
Fidelity: 0.9987, Success Rate: 0.7301


In [9]:
# Test 5: Mixed noise
print("TEST 5: Mixed noise")
analyze(0.9, 0.3/np.sqrt(3), 0.3/np.sqrt(3), 0.3/np.sqrt(3))

TEST 5: Mixed noise
Fidelity: 0.9007, Success Rate: 0.8129


## Summary

The `check_psip_psin` LOCC protocol works as follows:

1. **CNOT from data qubit to ancilla qubit** on both Alice's and Bob's sides
2. **Measure the ancilla qubits** (qubits 0 and 3)
3. **Post-select on XOR = 0** (both measurements give same result)

### Expected behavior:
- **φ+ and φ- noise**: These are in the "φ-type" family. When post-selecting on XOR=0, both φ+ and φ- components are **preserved/boosted**.
- **ψ+ and ψ- noise**: These are in the "ψ-type" family. When post-selecting on XOR=0, both ψ+ and ψ- components are **suppressed** (they contribute to the XOR=1 outcomes that are discarded).

This is a form of **entanglement distillation** that separates φ-type and ψ-type errors.